# Ch 18 State Space Models

## 18.1 Introduction

* hidden state가 continuous인 점만 빼고는 HMM과 거의 동일 
* 구성 
  * state transition model with system-error
  * observation model with observation-error
  
![](./images/ch18/01.png)   
  
  
* 주요 과제
  * exact/approximate inference
  * learning : estimate model parameters
* linear-gaussian SSM (LG-SSM)
  * easy case
    * hard case : non-linear case
  * support exact inference
    * kalman filtering
  
![](./images/ch18/02.png)

## 18.2 Application
### 18.2.1 SSM for object tracking

* tracking object by kalman filter from noisy measurements
* example : random acceleration model
  * object moving in 2D-space
  * move at constant velocity
  * perturbed by random gaussian noise, ex. wind
    * velocity jump by system noise
* model
  * state vector : position and velocity
  * observation vector : position only
  * LG-SSM assumption

![](./images/ch18/03.png)  
![](./images/ch18/04.png)  
![](./images/ch18/05.png)  

* initial belief and notation

![](./images/ch18/06.png)  


* estimation result
  * a) raw observation 은 measurement error 때문에 매우 noisy하다.
  * b) kalman filtering 적용 결과 
    * smmoth 해졌다. 
    * 원은 p(z|y), 즉 posterior가 gaussian임을 의미, 원 중심은 posterior mode에 해당
  * c) kalman smoothing 적용 결과                                

![](./images/ch18/07.png)  



### 18.2.2 Robotic SLAM (simultaneous localization and mapping)


* robot moving around unknown 2d world
* learn a map and keep track of its location within that map
* model
  * state vector
    * 로봇의 위치 at time t
    * L개의 landmark의 location at time t
      * fixed, no system error
  * observation
    * measurement of distances from robot location to the set of closest landmark
* How it works
  * inital guess about robot and landmark location
  * move arount and find some of landmark
  * measurement and update prior belief

![](./images/ch18/08.png)  


* Close-loop 현상
  * over time, uncertainty in robot location increase due to system-error
  * but when robot return to a familiar location, its uncertainty will decrease agamin
  * with smoothing, this shrink propagate back to the past trajectory
  
![](./images/ch18/09.png)  

* Convert to undirected graphical model
  * due to sparcity of posterior precision matrix
  * zero in matrix means no edges in UGM
  * at prior, only diagonal terms non-zero => UGM as disconnected graph
  * as robot moves, correlation between near-by landmarks induced
  
![](./images/ch18/10.png) 


* Entanglement problem
  * estimated landmark locations are not independent
  * because all depends on estimated robot locations
    * not d-seperated by observation : L1-Y1-X1-Y1-L2 경로
  * over time, precision matrix denser
  * hard to perform exact inference, O(K^3)
  * solution 
    * dynamically prune out ewak edegs ( junction tree in section 20.4 )
    * Fast SLAM : combination of kalman filter and particle filtering ( section 23.6.3 )

    
  

## 18.3 Inference in LG-SSM

* on-line case => analgous to the forward algorithm for HMMs
  * predict and correct
* off-line case => analgous to the forward-backward algorithm for HMMs

### 18.3.1 Kalman filtering algorithm

* 기본 idea
  * predict : 현재까지의 정보로 다음 time-step을 예측 
  * correct : 실제 관측된 사실을 바탕으로 잘못된 예측 모형을 바로잡음 ( error-based correction )
    * 사실상 강화학습의 TD-based 와 동일한 컨셉
    
* marginal posterior at time t

![](./images/ch18/11.png) 

* prediction-step

![](./images/ch18/12.png) 

* update by bayes rule
  * bayes rule : P(A|B) = ( P(B)*P(B|A) ) / (P(A)

![](./images/ch18/13.png) 


* error-signal
  * true measurement - guessed measurement

![](./images/ch18/14.png) 

* kalman gain factor
  * 역할 : update rule에서 일종의 learning rate에 해당
  * 이것도 정의는 아니고 원래 유도되는 것인데, 책에서는 그냥 define 으로 표기
  * prediction error를 minimize하는 objective를 설정해서 유도할 수 있음
  * 보다 직관적인 유도 과정은 다음 비디오 playlist 참고
    * https://www.youtube.com/playlist?list=PLsrI8QPOTtbbxi-Y5FyLOIX8dBmOtQMoU

![](./images/ch18/15.png) 

* final update rule
  * mean update만 생각하면 일종의 gradient-descent와 유사
  * kalman gain factor가 일종의 time-varying learning rate가 됨 
    * ratio between prior and measurement error
    * strong prior and/or noisy measurement => small learning rate
    * weak prior and precise measurement => large learning rate

![](./images/ch18/16.png) 


* posterior predictive
  * usefule for time-serires forcasting

![](./images/ch18/18.png) 

* log-likelihood of specific measurement sequences

![](./images/ch18/17.png) 

# Kalman filter의 시간 복잡도와 SMM의 Application 등은 다음 발표시간에 보충